### Домашнее задание к теме:   
**Коллаборативная фильтрация**

1. Использовать датасет [MovieLens](https://grouplens.org/datasets/movielens/latest/)  
2. Использовать любые модели  
3. Добиться целевого значения метрики RMSE на тестовой выборке 0.87 и ниже.

In [1]:
#pip install scikit-surprise

Намаялся с этим инсталлятором, помогло conda install -c conda-forge scikit-surprise через командную строку!

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [3]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [4]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
movies.shape, ratings.shape

((9742, 3), (100836, 4))

In [8]:
movies_with_ratings = pd.merge(movies, ratings, on='movieId', how='outer')
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [9]:
movies_with_ratings.isnull().any()

movieId      False
title        False
genres       False
userId        True
rating        True
timestamp     True
dtype: bool

In [10]:
movies_with_ratings.drop(columns=['timestamp'], inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5


In [11]:
movies_with_ratings.shape

(100854, 5)

In [12]:
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.shape

(100836, 5)

In [13]:
movies_with_ratings.isnull().any().any()

False

In [14]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [15]:
dataset.shape

(100836, 3)

In [16]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

Делим данные.

In [17]:
trainset, testset = train_test_split(data, test_size=0.2)

**item-based** подход

In [18]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [19]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8795


0.8794565030942215

RMSE выше требуемого

Флажок ставим на **user-based** подход

In [20]:
algo_ubased = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo_ubased.fit(trainset)
test_pred_ubased = algo_ubased.test(testset)
accuracy.rmse(test_pred_ubased, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8924


0.8924432738878137

По-прежнему RMSE высок

Алгоритм SVD

In [21]:
from surprise import SVD
algo_svd = SVD()
algo_svd.fit(trainset)
test_pred_svd = algo_svd.test(testset)
accuracy.rmse(test_pred_svd, verbose=True)

RMSE: 0.8683


0.8683206587665451

RMSE ниже 0.87